In [2]:
print(1)

1


In [1]:
import mediapipe as mp
import numpy as np
import time
import cv2
import torch

In [45]:
def extractLandmark(landmarks,landmark_lists : list)->list:

    landmark_list = []
    for lm in landmarks.landmark:
        landmark_list.append([lm.x, lm.y,lm.z])
    landmark_lists.append(landmark_list)

In [46]:

def landmarklist(results,lt,rt,ft,pt):
    
    

    if results.left_hand_landmarks:
        extractLandmark(results.left_hand_landmarks,lt) 
    else:
        lt.append(np.zeros([21,3]))
    if results.right_hand_landmarks:
        extractLandmark(results.right_hand_landmarks,rt)
    else:
        rt.append(np.zeros([21,3]))
    if results.face_landmarks:
        extractLandmark(results.face_landmarks,ft)
    else:
        ft.append(np.zeros([468, 3]))
    if results.pose_landmarks:
        extractLandmark(results.pose_landmarks,pt)
    else:
        pt.append(np.zeros([33, 3]))

    return lt, rt, ft, pt

In [26]:

holistic.close()
cap.release()
cv2.destroyAllWindows()

In [47]:
"""
이거 쓰세용     

"""
keypoints = []
left_hands = []
mp_holistic = mp.solutions.holistic
video_hol = mp.solutions.holistic.Holistic(static_image_mode=False,upper_body_only=False,smooth_landmarks=True,
                                           min_detection_confidence=0.5,min_tracking_confidence=0.5)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# For webcam input:
holistic = mp_holistic.Holistic(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture("./video/KETI_SL_0000006020.avi")

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# cap.set(cv2.CAP_PROP_FPS, 1)

total_frames = cap.get(7)
print(total_frames)

print(cap.get(cv2.CAP_PROP_FPS))

frame_rate = 10
prev = 0
i = 0
left_hand_lists = []
right_hand_lists = []
face_lists = []
pose_lists = []

start = time.time()
while cap.isOpened():
#     time_elapsed = time.time() - prev
    success, image = cap.read()

#     if time_elapsed > 1./frame_rate:
#         prev = time.time()
    # print(type(image))
    if not success:
        print("Ignoring empty camera frame.")
    # If loading a video, use 'break' instead of 'continue'.
        break

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic.process(image)
    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
      image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
      
    
    landmarklist(results,left_hand_lists ,right_hand_lists ,face_lists ,pose_lists)

        
        
    cv2.imshow('MediaPipe Holistic', image)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
holistic.close()
cap.release()
cv2.destroyAllWindows()
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

-1.0
30.0
time : 14.50534987449646


In [117]:

lt = torch.FloatTensor(left_hand_lists)
rt = torch.FloatTensor(right_hand_lists)
ft = torch.FloatTensor(face_lists)
pt = torch.FloatTensor(pose_lists)


print(
torch.FloatTensor(left_hand_lists).shape,
torch.FloatTensor(right_hand_lists).shape,
torch.FloatTensor(face_lists).shape,
torch.FloatTensor(pose_lists).shape
)

torch.Size([108, 21, 3]) torch.Size([108, 21, 3]) torch.Size([108, 468, 3]) torch.Size([108, 33, 3])


In [113]:
def pipe(t):
    t = t[:15*(t.shape[0]//15)]
    return t.reshape(t.shape[0]//15,15,t.shape[1]*t.shape[2])

In [118]:
lt = pipe(lt)
rt = pipe(rt)
ft = pipe(ft)
pt = pipe(pt)


In [119]:

print(lt.shape, rt.shape, ft.shape, pt.shape)

torch.Size([7, 15, 63]) torch.Size([7, 15, 63]) torch.Size([7, 15, 1404]) torch.Size([7, 15, 99])


In [121]:
class Net(torch.nn.Module):
    def __init__(self,seq_size, hidden_size, output_size):
        super(Net, self).__init__()
        # self.lt = torch.nn.RNN((None,21,3), hidden_size, batch_first=True) # RNN 셀 구현
        # self.rt = torch.nn.RNN((None,21,3), hidden_size, batch_first=True) # RNN 셀 구현
        # self.ft = torch.nn.RNN((None,468,3), hidden_size, batch_first=True) # RNN 셀 구현
        # self.pt = torch.nn.RNN((None,33,3), hidden_size, batch_first=True) # RNN 셀 구현
#         [None,21,3]
# (None,21,3)
# (None,468,3)
# (None,33,3)
        self.lt = torch.nn.RNN(21*3, hidden_size) # RNN 셀 구현
        self.rt = torch.nn.RNN(21*3, hidden_size) # RNN 셀 구현
        self.ft = torch.nn.RNN(468*3, hidden_size) # RNN 셀 구현
        self.pt = torch.nn.RNN(33*3, hidden_size) # RNN 셀 구현
        self.dnnlt = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
        self.dnnrt = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
        self.dnnft = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
        self.dnnpt = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
        self.outDnn = torch.nn.Linear(output_size,50,bias=True)
    def forward(self,lt,rt,ft,pt ): # 구현한 RNN 셀과 출력층을 연결
        rlt, _status =self.lt(lt)
        rrt, _status =self.rt(rt)
        rft, _status =self.ft(ft)
        rpt, _status =self.pt(pt)

        dlt= self.dnnlt(rlt)
        drt= self.dnnrt(rrt)
        dft= self.dnnft(rft)
        dpt= self.dnnpt(rpt)

        fullDnn = torch.cat((dlt, drt, dft, dpt),dim=1)
        out = self.outDnn(fullDnn)

        return out

In [122]:

net = Net(lt.shape[0], 60,200)
net.forward

<bound method Net.forward of Net(
  (lt): RNN(63, 60)
  (rt): RNN(63, 60)
  (ft): RNN(1404, 60)
  (pt): RNN(99, 60)
  (dnnlt): Linear(in_features=60, out_features=200, bias=True)
  (dnnrt): Linear(in_features=60, out_features=200, bias=True)
  (dnnft): Linear(in_features=60, out_features=200, bias=True)
  (dnnpt): Linear(in_features=60, out_features=200, bias=True)
  (outDnn): Linear(in_features=200, out_features=50, bias=True)
)>

In [123]:
print(lt.shape)

net(lt,rt,ft,pt)

torch.Size([7, 15, 63])
torch.Size([7, 60, 200])


tensor([[[-0.0037,  0.0078, -0.0582,  ...,  0.0531,  0.1692, -0.0409],
         [-0.0037,  0.0078, -0.0582,  ...,  0.0531,  0.1692, -0.0409],
         [-0.0037,  0.0078, -0.0582,  ...,  0.0531,  0.1692, -0.0409],
         ...,
         [-0.0384,  0.1855,  0.1462,  ..., -0.0618,  0.3647, -0.0924],
         [-0.0384,  0.1852,  0.1469,  ..., -0.0610,  0.3639, -0.0942],
         [-0.0390,  0.1854,  0.1472,  ..., -0.0618,  0.3645, -0.0920]],

        [[-0.0221,  0.0268, -0.0264,  ...,  0.0541,  0.2037, -0.0179],
         [-0.0221,  0.0268, -0.0264,  ...,  0.0541,  0.2037, -0.0179],
         [-0.0221,  0.0268, -0.0264,  ...,  0.0541,  0.2037, -0.0179],
         ...,
         [ 0.0265,  0.1980,  0.1420,  ..., -0.0070,  0.3368, -0.0846],
         [ 0.0247,  0.2004,  0.1554,  ...,  0.0010,  0.3399, -0.0881],
         [ 0.0266,  0.1964,  0.1472,  ..., -0.0046,  0.3299, -0.0889]],

        [[ 0.0906, -0.0283,  0.1232,  ...,  0.0201,  0.1985, -0.1995],
         [ 0.0909, -0.0277,  0.1241,  ...,  0

In [100]:
tmp = lt
tmp.reshape([108,63])
t = tmp[:105].reshape([105,63])
t.shape
t = t.reshape(lt.shape[0]//15,15,lt.shape[1]*lt.shape[2])
print(t.shape)


torch.Size([7, 15, 63])


매 frame마다 tensor를 만들고

실제 모델 실행시 90frame을 넣는다라고 가정을하면

tensor를 만드는거는 값을 직접입력하는 방식으로 진행하고 

만든 tensor를 리스트에 append한다. 

그 이후 b = torch.Tensor(90,size,3)

torch.cat(tensorlist[-90:],out = b)
이런식으로 뽑는다



